In [1]:
import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import label_wav
from libs import input_data
from libs import models
from tensorflow.python.platform import gfile

In [2]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('results_dir','..\\..\\results','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
flags.DEFINE_string('predictions_dir','..\\..\\predictions','Predictions Directory')
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
FLAGS=flags.FLAGS

### Variables

In [3]:
model_architecture='ds_cnn'
model_version='30000'

### Prepare Input Files

In [4]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [5]:
test_dir=os.path.join(FLAGS.data_dir,'test','audio','*.wav')

In [6]:
# Must be in sorted order
testing_files_list=sorted(gfile.Glob(test_dir))

In [7]:
testing_files_list

['..\\..\\_inputs\\raw\\test\\audio\\00b01445_nohash_0.wav',
 '..\\..\\_inputs\\raw\\test\\audio\\00f0204f_nohash_0.wav',
 '..\\..\\_inputs\\raw\\test\\audio\\00f0204f_nohash_1.wav',
 '..\\..\\_inputs\\raw\\test\\audio\\0a9f9af7_nohash_0.wav',
 '..\\..\\_inputs\\raw\\test\\audio\\0a9f9af7_nohash_1.wav',
 '..\\..\\_inputs\\raw\\test\\audio\\0ab3b47d_nohash_0.wav',
 '..\\..\\_inputs\\raw\\test\\audio\\0ab3b47d_nohash_1.wav']

### Predict 

In [8]:
labels_list=input_data.prepare_words_list(FLAGS.wanted_words.split(','))
graph=os.path.join(FLAGS.models_dir,model_architecture,'ckpt-'+model_version+'-verysmall-batched.pb')

In [9]:
pred_mat,pred=label_wav.label_wav_batched(testing_files_list,labels_list,graph,batch_size=1000)

W0121 21:16:56.033125 13716 deprecation.py:323] From ..\libs\label_wav.py:37: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
W0121 21:16:56.038109 13716 deprecation_wrapper.py:119] From ..\libs\label_wav.py:38: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.

W0121 21:16:56.067539 13716 deprecation_wrapper.py:119] From ..\libs\label_wav.py:94: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Number of Files 7
Processed 100.000000%


### Save the Predictions

In [10]:
save_folder='ds_cnn_verysmall_psuedo'

In [11]:
save_path=os.path.join(FLAGS.predictions_dir,save_folder)
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [12]:
f_names=[]
for f in testing_files_list:
    f_names.append(os.path.basename(f))
pred_df=pd.DataFrame({'fname':f_names,'label':pred})

In [13]:
pred_df.to_csv(os.path.join(save_path,'predictions.csv'),index=None)

In [14]:
np.save(os.path.join(save_path,'pred_mat.npy'),pred_mat)

In [15]:
pred_df

,fname,label
0,00b01445_nohash_0.wav,go
1,00f0204f_nohash_0.wav,go
2,00f0204f_nohash_1.wav,go
3,0a9f9af7_nohash_0.wav,go
4,0a9f9af7_nohash_1.wav,go
5,0ab3b47d_nohash_0.wav,go
6,0ab3b47d_nohash_1.wav,go


In [16]:
testing_files_list=''
for f in f_names:
    testing_files_list=testing_files_list+str(os.path.join('test','audio',f))+'\n'

In [17]:
with open('validation','w') as file:
    file.write(testing_files_list)